In [ ]:
import pyautogui
import cv2
import numpy as np
import easyocr

# Step 1: Take a screenshot of the left middle of the screen
def take_screenshot(region):
    screenshot = pyautogui.screenshot(region=region)
    screenshot = np.array(screenshot)
    return cv2.cvtColor(screenshot, cv2.COLOR_RGB2BGR)

# Step 2: Automatically detect the grid
def detect_grid(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    edged = cv2.Canny(blurred, 30, 150)

    # Find contours in the edge map
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Loop over the contours to find the grid
    for contour in contours:
        # Get the bounding box for each contour
        x, y, w, h = cv2.boundingRect(contour)
        
        # Assuming the grid is roughly square-shaped and larger than a threshold size
        aspect_ratio = w / float(h)
        if 0.9 <= aspect_ratio <= 1.1 and w > 100 and h > 100:
            return image[int(y*1.05):int(y*0.95)+h, int(x*1.125):int(x*0.875)+w], (x, y, w, h)
    
    return None, None

# Step 3: Convert the grid into a matrix
def grid_to_matrix(grid):
    reader = easyocr.Reader(['en'])
    grid_size = 9
    cell_width = grid.shape[1] // grid_size
    cell_height = grid.shape[0] // grid_size
    matrix = np.zeros((grid_size, grid_size), dtype=int)
    for row in range(grid_size):
        for col in range(grid_size):
            # Calculate the coordinates of the cell
            x_start = col * cell_width
            y_start = row * cell_height
            x_end = (col + 1) * cell_width
            y_end = (row + 1) * cell_height

            # Crop the cell from the image
            cell = grid[y_start:y_end, x_start:x_end]

            # Check the color of the cell
            mean_color = np.mean(cell)

            if mean_color > 150:  # If the cell is predominantly white
                matrix[row, col] = 0
            else:  # If the cell is predominantly black
                result = reader.readtext(cell, detail=0)
                if result:
                    matrix[row, col] = int(result[0])
                else:
                    matrix[row, col] = -1
    return matrix
    
    
    
# Define the region for the left middle of the screen
screen_width, screen_height = pyautogui.size()
region = (0, 0, screen_width // 2, screen_height)

# Take the screenshot
screenshot = take_screenshot(region)

# Detect the grid
grid, grid_region = detect_grid(screenshot)

if grid is not None:
    # Save the detected grid image to verify
    print(f"Grid detected at region: {grid_region}")
    cv2.imwrite("grid.png", grid)
    # Convert the grid into a matrix
    matrix = grid_to_matrix(grid)
    print(matrix)
else:
    print("No grid detected")